In [1]:
import pandas as pd
import re, math
import datetime as dt 
import numpy as np
import collections
from collections import Counter
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\valen\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
times= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\timesData.csv')
shanghai= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\shanghaiData.csv')
attainment= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\educational_attainment_supplementary_data.csv')
school_country= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\school_and_country_table.csv')
expenditure= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\education_expenditure_supplementary_data.csv',sep=',', lineterminator='\r')
# lineterminator='\r' : Character to break file into lines. \r means carriage return. It means that the cursor should go back to the beginning of the line.
cwur= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\cwurData.csv')


In [3]:
times.head(2)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011


In [4]:
shanghai.head(2)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005


In [5]:
cwur.head(2)

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012


In [6]:
attainment.head(2)


,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN


In [7]:
school_country.head(2)


,school_name,country
0,Harvard University,United States of America
1,California Institute of Technology,United States of America


In [8]:
expenditure.head(2)

,country,institute_type,direct_expenditure_type,1995,2000,2005,2009,2010,2011
0,OECD Average,All Institutions,Public,4.9,4.9,5.0,5.4,5.4,5.3
1,Australia,All Institutions,Public,4.5,4.6,4.3,4.5,4.6,4.3


In [30]:
#times_unique = pd.Series(times['university_name'].unique(), name='university_name')

In [32]:
#Similarità tra i nomi delle università
compare = pd.MultiIndex.from_product([times['university_name'].unique(),
                                      times['university_name'].unique()]).to_series()

In [33]:
def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup)],
                     ['ratio', 'token'])

In [34]:
compare.apply(metrics)

ratio  \
Harvard University           Harvard University                       100   
                             California Institute of Technology        31   
                             Massachusetts Institute of Technology     29   
                             Stanford University                       76   
                             Princeton University                      63   
...                                                                   ...   
Yokohama National University Xidian University                         58   
                             Yeungnam University                       60   
                             Yıldız Technical University               58   
                             Yokohama City University                  81   
                             Yokohama National University             100   

                                                                    token  
Harvard University           Harvard University                       100  
                             California Institute of Technology        31  
                             Massachusetts Institute of Technology     18  
                             Stanford University                       76  
                             Princeton University                      63  
...                                                                   ...  
Yokohama National University Xidian University                         53  
                             Yeungnam University                       60  
                             Yıldız Technical University               62  
                             Yokohama City University                  81  
                             Yokohama National University             100  

[669124 rows x 2 columns]

# **1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes**

In [9]:
times.year.describe()

count    2603.000000
mean     2014.075682
std         1.685733
min      2011.000000
25%      2013.000000
50%      2014.000000
75%      2016.000000
max      2016.000000
Name: year, dtype: float64

In [10]:
times['university_name'].unique()

array(['Harvard University', 'California Institute of Technology',
       'Massachusetts Institute of Technology', 'Stanford University',
       'Princeton University', 'University of Cambridge',
       'University of Oxford', 'University of California, Berkeley',
       'Imperial College London', 'Yale University',
       'University of California, Los Angeles', 'University of Chicago',
       'Johns Hopkins University', 'Cornell University',
       'ETH Zurich – Swiss Federal Institute of Technology Zurich',
       'University of Michigan', 'University of Toronto',
       'Columbia University', 'University of Pennsylvania',
       'Carnegie Mellon University', 'University of Hong Kong',
       'University College London', 'University of Washington',
       'Duke University', 'Northwestern University',
       'University of Tokyo', 'Georgia Institute of Technology',
       'Pohang University of Science and Technology',
       'University of California, Santa Barbara',
       'Universi

In [11]:
times['university_name'].nunique()

818

In [12]:
times_most_recent = times.loc[times.groupby('university_name')['year'].idxmax()]
times_most_recent.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016


In [13]:
times_most_recent['year'].describe()

count     818.000000
mean     2015.951100
std         0.380111
min      2011.000000
25%      2016.000000
50%      2016.000000
75%      2016.000000
max      2016.000000
Name: year, dtype: float64

In [14]:
len(times_most_recent)

818

In [15]:
times_least_recent = times.loc[times.groupby('university_name')['year'].idxmin()]
times_least_recent.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012


In [16]:
times_least_recent['year'].describe()

count     818.00000
mean     2013.56846
std         2.17774
min      2011.00000
25%      2012.00000
50%      2013.00000
75%      2016.00000
max      2016.00000
Name: year, dtype: float64

In [17]:
len(times_least_recent)

818

In [18]:
times[times['university_name'] == 'AGH University of Science and Technology']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016


# **2. For each university, compute the improvement in income between the least recent and the most recent data points**

Income: sul sito https://www.timeshighereducation.com/news/ranking-methodology-2016 viene indicato quanto segue:
"Industry income (knowledge transfer): 2.5%
A university’s ability to help industry with innovations, inventions and consultancy has become a core mission of the contemporary global academy. This category seeks to capture such knowledge-transfer activity by looking at how much research income an institution earns from industry (adjusted for PPP), scaled against the number of academic staff it employs.

The category suggests the extent to which businesses are willing to pay for research and a university’s ability to attract funding in the commercial marketplace – useful indicators of institutional quality".


In [19]:
#ci sono dei missing sul campo income
times.groupby('income').size()

income
-        218
100.0     68
24.2       5
24.5       2
24.6       1
        ... 
99.5       5
99.6       2
99.7       4
99.8      10
99.9      14
Length: 613, dtype: int64

In [20]:
#Replacement di - con 00.00  
times_most_recent['income'].replace('-', '00.00', inplace=True)
times_least_recent['income'].replace('-', '00.00', inplace=True)

In [21]:
times_most_recent['income'] = times_most_recent['income'].astype(float)
times_least_recent['income'] = times_least_recent['income'].astype(float)

In [22]:
#merge dei df most recent e least recent 
merged_least_most = times_most_recent.merge(times_least_recent, on = 'university_name', suffixes=('_most_recent', '_least_recent')).reset_index()
merged_least_most['diff_year'] = merged_least_most['year_most_recent'] - merged_least_most['year_least_recent']

In [23]:
merged_least_most[merged_least_most['diff_year'] == 0]  

,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,research_least_recent,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year
0,0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,...,3.7,35.7,0.0,-,"35,569",17.0,1%,-,2016,0
5,5,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,-,...,11.0,15.3,28.7,-,"40,633",15.6,1%,71 : 29,2016,0
6,6,251-300,Aix-Marseille University,France,36.7,63.0,22.1,64.9,33.1,-,...,22.1,64.9,33.1,-,"71,749",45.5,13%,61 : 39,2016,0
7,7,601-800,Ajou University,South Korea,19.5,20.0,11.9,23.9,45.7,-,...,11.9,23.9,45.7,-,"12,706",11.3,2%,33 : 67,2016,0
9,9,601-800,Alexandru Ioan Cuza University,Romania,24.9,46.9,13.6,7.0,28.2,-,...,13.6,7.0,28.2,-,"25,724",33.0,5%,63 : 37,2016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,805,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,...,10.9,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016,0
806,806,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,...,10.2,36.4,37.9,-,"4,122",3.7,3%,NaN,2016,0
807,807,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,...,16.0,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016,0
811,811,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,...,7.6,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016,0


In [24]:
#Drop dei records per i quali abbiamo lo stessa osservazione (per stessa univ. anno least e anno most recent uguali)
#merged_least_most = merged_least_most[merged_least_most['diff_year'] != 0]  
#merged_least_most

In [25]:
merged_least_most['delta'] = merged_least_most['income_most_recent']- merged_least_most['income_least_recent']
merged_least_most

,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year,delta
0,0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,...,35.7,0.0,-,"35,569",17.0,1%,-,2016,0,0.0
1,1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,...,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012,4,7.3
2,2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,...,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012,4,-0.3
3,3,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,...,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011,5,6.8
4,4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012,4,-4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,813,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,...,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011,5,6.4
814,814,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,...,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011,5,5.6
815,815,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,...,91.4,0.0,69.5,"2,429",4.8,30%,18 : 82,2011,5,82.3
816,816,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,...,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011,5,27.4


In [26]:
#migliorate
merged_least_most[merged_least_most['delta'] > 0].sort_values(by =['delta'], ascending=False)


,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year,delta
788,788,=47,Wageningen University and Research Center,Netherlands,52.6,70.3,66.9,87.8,100.0,70.0,...,53.0,0.0,51.9,"9,248",17.0,21%,56 : 44,2011,5,100.0
438,438,53,Technical University of Munich,Germany,61.0,63.8,66.0,80.1,99.2,69.4,...,71.2,0.0,56.9,"35,565",31.5,20%,33 : 67,2011,5,99.2
632,632,=65,University of Minnesota,United States of America,53.5,35.3,61.0,88.0,98.5,65.9,...,76.4,0.0,65.6,"46,825",18.0,13%,52 : 48,2011,5,98.5
247,247,=88,Maastricht University,Netherlands,37.5,95.5,47.7,82.7,95.4,59.9,...,53.9,0.0,41.5,"15,626",18.9,48%,56 : 44,2012,4,95.4
529,529,=120,University of Cape Town,South Africa,34.9,80.2,38.7,85.6,92.7,56.1,...,82.8,0.0,56.1,"20,040",12.1,18%,53 : 47,2011,5,92.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,201-250,Chalmers University of Technology,Sweden,36.6,77.6,35.3,52.2,79.5,-,...,36.3,79.3,-,"8,605",11.6,15%,30 : 70,2012,4,0.2
550,550,=185,University of Dundee,United Kingdom,25.6,78.5,26.9,90.2,49.5,49.9,...,79.3,49.3,52.2,"11,628",15.3,25%,60 : 40,2011,5,0.2
385,385,351-400,San Diego State University,United States of America,20.3,25.9,27.0,62.7,29.3,-,...,61.1,29.2,-,"27,420",31.7,5%,57 : 43,2014,2,0.1
575,575,=99,University of Göttingen,Germany,49.3,56.4,54.5,72.7,31.8,58.0,...,92.5,31.7,67.0,"25,581",25.6,12%,51 : 49,2011,5,0.1


In [27]:
#peggiorate
merged_least_most[merged_least_most['delta'] < 0]

,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year,delta
2,2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,...,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012,4,-0.3
4,4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012,4,-4.2
8,8,601-800,Alexandria University,Egypt,20.3,33.7,8.2,14.1,29.7,-,...,99.8,36.0,51.6,"127,431",23.3,1%,46 : 54,2011,5,-6.3
23,23,501-600,Auburn University,United States of America,30.2,27.9,19.4,21.0,37.2,-,...,10.3,47.3,-,"22,386",17.6,4%,51 : 49,2012,4,-10.1
27,27,146,Autonomous University of Barcelona,Spain,40.3,50.3,40.0,83.8,34.9,53.8,...,57.9,37.0,-,"30,538",12.3,10%,59 : 41,2012,4,-2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,772,351-400,Université Libre de Bruxelles,Belgium,26.1,84.5,28.3,47.9,39.1,-,...,53.5,52.8,-,"23,819",26.1,32%,54 : 46,2012,4,-13.7
779,779,154,VU University Amsterdam,Netherlands,28.4,54.9,37.1,90.7,60.1,52.5,...,58.5,81.4,52.3,"23,280",16.3,6%,56 : 44,2011,5,-21.3
780,780,87,Vanderbilt University,United States of America,47.8,29.1,44.2,95.7,63.2,60.1,...,78.1,84.2,65.9,"12,161",3.6,10%,52 : 48,2011,5,-21.0
799,799,351-400,Wuhan University of Technology,China,16.2,21.4,8.0,75.8,57.8,-,...,78.1,58.7,-,"50,260",14.9,2%,31 : 69,2014,1,-0.9


In [28]:
#invariate
merged_least_most[merged_least_most['delta'] == 0]

,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year,delta
0,0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,...,35.7,0.0,-,"35,569",17.0,1%,-,2016,0,0.0
5,5,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,-,...,15.3,28.7,-,"40,633",15.6,1%,71 : 29,2016,0,0.0
6,6,251-300,Aix-Marseille University,France,36.7,63.0,22.1,64.9,33.1,-,...,64.9,33.1,-,"71,749",45.5,13%,61 : 39,2016,0,0.0
7,7,601-800,Ajou University,South Korea,19.5,20.0,11.9,23.9,45.7,-,...,23.9,45.7,-,"12,706",11.3,2%,33 : 67,2016,0,0.0
9,9,601-800,Alexandru Ioan Cuza University,Romania,24.9,46.9,13.6,7.0,28.2,-,...,7.0,28.2,-,"25,724",33.0,5%,63 : 37,2016,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,805,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,...,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016,0,0.0
806,806,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,...,36.4,37.9,-,"4,122",3.7,3%,NaN,2016,0,0.0
807,807,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,...,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016,0,0.0
811,811,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,...,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016,0,0.0


# **3. Find the university with the largest increase computed in the previous point**


In [29]:
univ_max_impr = merged_least_most.delta.idxmax()

In [30]:
merged_least_most[merged_least_most.index == univ_max_impr]

,index,world_rank_most_recent,university_name,country_most_recent,teaching_most_recent,international_most_recent,research_most_recent,citations_most_recent,income_most_recent,total_score_most_recent,...,citations_least_recent,income_least_recent,total_score_least_recent,num_students_least_recent,student_staff_ratio_least_recent,international_students_least_recent,female_male_ratio_least_recent,year_least_recent,diff_year,delta
788,788,=47,Wageningen University and Research Center,Netherlands,52.6,70.3,66.9,87.8,100.0,70.0,...,53.0,0.0,51.9,"9,248",17.0,21%,56 : 44,2011,5,100.0


# **4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range**

I ranking sono presenti nei ds di times, cwur e shanghai

In [31]:
shanghai.year.describe()

count    4897.000000
mean     2009.658566
std         3.197576
min      2005.000000
25%      2007.000000
50%      2009.000000
75%      2012.000000
max      2015.000000
Name: year, dtype: float64

In [32]:
#ci sono alcune università uguali ma con descrizione diversa nei diversi anni, elimino i caratteri speciali per limitare questo problema
shanghai['university_name'] = shanghai['university_name'].str.replace('[^A-Za-z0-9]', ' ')

In [33]:
#world_rank espresso in range nel ds shanghai e total score non valorizzato
shanghai[shanghai['world_rank'].str.contains("-")]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
100,101-152,Aarhus University,2,NaN,15.4,19.3,7.9,22.3,41.6,22.4,2005
101,101-152,Arizona State University Tempe,54-71,NaN,0.0,14.4,20.8,26.3,41.9,17.5,2005
102,101-152,Baylor College of Medicine,54-71,NaN,0.0,0.0,17.6,34.5,44.0,24.9,2005
103,101-152,Catholic University of Leuven,1-4,NaN,0.0,0.0,19.2,16.0,48.7,23.1,2005
104,101-152,Catholic University of Louvain,1-4,NaN,14.0,13.9,13.6,8.3,44.7,26.9,2005
...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4893,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4894,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4895,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015


In [34]:
shanghai = shanghai.fillna(0)

In [35]:
shanghai['total_score'] = shanghai['total_score'].astype(float)

In [36]:
#http://www.shanghairanking.com/ARWU-Methodology-2016.html --> metodologia per il calcolo dello score
#aggiungo una colonna al df con i nuovi punteggi grezzi non ancora aggiustati rispetto al punteggio max per ciascun anno (il primo in classifica vale 100, gli altri punteggi vengono aggiustati in funzione di questo)
shanghai['new_total_score'] = (shanghai['alumni']*10/100+shanghai['award']*20/100+shanghai['hici']*20/100+
                               shanghai['ns']*20/100+shanghai['pub']*20/100+shanghai['pcp']*10/100)


In [37]:
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,97.24
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,71.51
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,71.39
3,4,University of California Berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,70.75
4,5,Massachusetts Institute of Technology MIT,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,68.18


In [38]:
#estrazione punteggi max per ciascun anno e creazione di un dizionario con il peso per ciascuno score max relativo ad ogni anno
top_scores_year = shanghai.groupby('year')['new_total_score'].max().reset_index()
top_scores_year['weight'] = 100/top_scores_year['new_total_score']
top_scores_year_dict = pd.Series(top_scores_year.weight.values,index=top_scores_year.year).to_dict()
top_scores_year_dict

{2005: 1.0283833813245578,
 2006: 1.0271158586688578,
 2007: 1.027749229188078,
 2008: 1.026588645929576,
 2009: 1.0258514567090684,
 2010: 1.031778786628147,
 2011: 1.0303967027305514,
 2012: 1.0297600659046442,
 2013: 1.0282776349614395,
 2014: 1.0253255408592228,
 2015: 1.0239606799098915}

In [39]:
#aggiunta della colonna contente il peso nel ds di shanghai
shanghai['weight_new_total_score'] = shanghai['year'].map(top_scores_year_dict)
#visualizzazione di un'università
shanghai[shanghai['university_name'] == 'Aarhus University']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
100,101-152,Aarhus University,2,0.0,15.4,19.3,7.9,22.3,41.6,22.4,2005,22.00,1.028383
601,102-150,Aarhus University,2,0.0,14.8,18.9,7.7,23.8,42.7,24.6,2006,22.56,1.027116
1101,102-150,Aarhus University,2,0.0,14.4,18.9,7.4,24.3,40.8,23.7,2007,22.09,1.027749
1603,93,Aarhus University,2,24.5,13.7,18.9,7.3,24.5,48.2,27.0,2008,23.85,1.026589
2109,97,Aarhus University,2,24.2,13.4,18.9,7.3,24.0,47.6,26.6,2009,23.56,1.025851
2611,98,Aarhus University,2,24.2,13.1,18.9,7.2,23.5,48.4,25.4,2010,23.45,1.031779
3099,86,Aarhus University,2,25.4,13.2,22.1,7.2,24.3,49.3,27.9,2011,24.69,1.030397
3599,86,Aarhus University,2,25.5,12.1,22.8,7.2,23.9,50.8,26.6,2012,24.81,1.029760
3878,81,Aarhus University,2.0,26.0,12.0,22.2,7.2,25.6,50.9,28.5,2013,25.23,1.028278
3970,74,Aarhus University,2,27.4,11.8,22.1,12.3,26.7,51.9,29.8,2014,26.76,1.025326


In [40]:
#score aggiustato per il peso
shanghai['new_total_score'] = (shanghai['new_total_score']*shanghai['weight_new_total_score']).round(1)

In [41]:
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383
3,4,University of California Berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383
4,5,Massachusetts Institute of Technology MIT,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,University of Trieste,11-20,0.0,0.0,0.0,5.0,10.9,25.1,20.1,2015,10.5,1.023961
4893,401-500,University of Zaragoza,9-13,0.0,0.0,0.0,7.6,5.1,33.3,13.1,2015,10.8,1.023961
4894,401-500,Utah State University,126-146,0.0,13.6,0.0,3.6,10.8,25.1,15.5,2015,11.1,1.023961
4895,401-500,Vienna University of Technology,4-6,0.0,0.0,0.0,0.0,12.2,28.8,22.9,2015,10.7,1.023961


In [42]:
shanghai = shanghai.groupby('year').apply(lambda x: x.sort_values('new_total_score', ascending= False )).reset_index(drop=True)
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383
3,4,University of California Berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383
4,5,Massachusetts Institute of Technology MIT,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,Okayama University,13-18,0.0,0.0,0.0,3.6,9.0,28.3,11.7,2015,9.6,1.023961
4893,401-500,The University of Tokushima,13-18,0.0,13.6,0.0,5.1,8.0,19.7,14.7,2015,9.6,1.023961
4894,401-500,University of Rennes 1,19-22,0.0,0.0,0.0,3.6,9.2,28.1,11.2,2015,9.5,1.023961
4895,401-500,Nara Institute of Science and Technology,13-18,0.0,0.0,0.0,13.3,13.5,13.9,11.4,2015,9.5,1.023961


In [43]:
#siccome nei records in cui sono presenti i world_rank non espressi in range sono presenti dei ranking ripetuti quando i total scores delle università sono uguali, creo un sottoindice per ciascun anno per tenere l'ordinamento originale ed assegno lo stesso posto in classifica se il punteggio del new_total_score è uguale per due università
shanghai['group_index_sha'] = shanghai.groupby('year').cumcount() + 1

In [44]:
shanghai['new_world_ranking_sha'] = shanghai['group_index_sha'] 

In [45]:
#per creare il nuovo ranking utilizziamo il new_total_rank 
for i in range(1, len(shanghai)):

    if shanghai.new_total_score.loc[i-1] != shanghai.new_total_score.loc[i]:
        shanghai['new_world_ranking_sha'].loc[i] = shanghai['group_index_sha'].loc[i]
        
    else:
        shanghai['new_world_ranking_sha'].loc[i] = shanghai.new_world_ranking_sha.loc[i-1]
    i = i+1

C:\Users\valen\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [46]:
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383,1,1
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383,2,2
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383,3,3
3,4,University of California Berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383,4,4
4,5,Massachusetts Institute of Technology MIT,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,401-500,Okayama University,13-18,0.0,0.0,0.0,3.6,9.0,28.3,11.7,2015,9.6,1.023961,496,492
4893,401-500,The University of Tokushima,13-18,0.0,13.6,0.0,5.1,8.0,19.7,14.7,2015,9.6,1.023961,497,492
4894,401-500,University of Rennes 1,19-22,0.0,0.0,0.0,3.6,9.2,28.1,11.2,2015,9.5,1.023961,498,498
4895,401-500,Nara Institute of Science and Technology,13-18,0.0,0.0,0.0,13.3,13.5,13.9,11.4,2015,9.5,1.023961,499,498


In [47]:
shanghai[shanghai['university_name']==0]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
3896,99,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,0.0,1.028278,99,99


In [48]:
shanghai.drop(shanghai.index[3896], inplace= True)

In [49]:
#creazione del df con i dati più recenti per ciascuna università
shanghai_most_recent = shanghai.loc[shanghai.groupby('university_name')['year'].idxmax()].reset_index()

In [50]:
shanghai_most_recent

,index,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
0,4793,301-400,Aalborg University,5,0.0,0.0,0.0,11.2,4.6,30.4,16.8,2015,11.2,1.023961,397,392
1,4820,401-500,Aalto University,4-6,0.0,0.0,0.0,0.0,10.4,33.4,17.0,2015,10.7,1.023961,424,424
2,4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,27.3,1.023961,73,73
3,4528,101-150,Aix Marseille University,5-6,0.0,13.6,0.0,15.2,20.3,50.1,27.6,2015,21.7,1.023961,132,132
4,4869,401-500,Aristotle University of Thessaloniki,2,0.0,0.0,0.0,4.9,1.5,34.0,16.8,2015,10.0,1.023961,473,471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,3489,401-500,Yamaguchi University,17-23,0.0,0.0,0.0,16.1,2.8,20.0,13.2,2011,9.4,1.030397,476,474
652,4661,201-300,Yeshiva University,79-102,0.0,0.0,0.0,6.3,21.7,31.5,21.6,2015,14.4,1.023961,265,263
653,4635,201-300,Yonsei University,2-5,0.0,0.0,0.0,8.6,4.9,52.1,21.4,2015,15.6,1.023961,239,237
654,4390,401-500,York University,19-21,0.0,0.0,0.0,0.0,10.1,30.6,12.4,2014,9.6,1.025326,494,491


In [51]:
shanghai_most_recent.year.describe()

count     656.000000
mean     2013.917683
std         2.338380
min      2005.000000
25%      2015.000000
50%      2015.000000
75%      2015.000000
max      2015.000000
Name: year, dtype: float64

In [52]:
shanghai_most_recent = shanghai_most_recent.sort_values(by =['year','new_world_ranking_sha'], ascending=True).reset_index(drop=True)
shanghai_most_recent[shanghai_most_recent['university_name']== 'Aarhus University']

,index,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
228,4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,27.3,1.023961,73,73


In [53]:
times= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\timesData.csv')

In [54]:
times.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011


In [55]:
times.dtypes

world_rank                 object
university_name            object
country                    object
teaching                  float64
international              object
research                  float64
citations                 float64
income                     object
total_score                object
num_students               object
student_staff_ratio       float64
international_students     object
female_male_ratio          object
year                        int64
dtype: object

In [56]:
#pulizia dei campi
times['income'].replace('-', '00.00', inplace=True)
times['international'].replace('-', '00.00', inplace=True)
times['world_rank'] = times['world_rank'].str.replace('=', '')
times['international_students'] = times['international_students'].str.replace('%', ' ')
times['income'] = times['income'].astype(float)
times['international'] = times['international'].astype(float)
times['international_students'] = times['international_students'].astype(float)

In [57]:
#https://www.timeshighereducation.com/news/ranking-methodology-2016
#La metodologia di calcolo del ranking è variata nel 2012
#tuttavia, ci sono solo tre univ che hanno come dato più recente il 2011 quindi usare questo metodo consente di essere più precisi. In alternativa,si potrebbe usare il punto medio.  ancora è stato valutato l'utilizzo del 
#Non è consigliabile usare l'ordinamento dei ranking presente nel ds perchè per i ranking espressi in range l'ordine nel ds è alfabetico sul nome università
#Come per il ds shanghai, se due univ hanno medesimo new total score, il new_world_rank è uguale
times['new_total_score'] = (times['teaching']*30/100+times['international']*7.5/100+times['research']*30/100+times['citations']*30/100+times['income']*2.5/100).round(1)

In [58]:
times = times.groupby('year').apply(lambda x: x.sort_values('new_total_score', ascending= False )).reset_index(drop=True)
times[times['year']==2012]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score
200,1,California Institute of Technology,United States of America,95.7,56.0,98.2,99.9,97.0,94.8,"2,243",6.9,27.0,33 : 67,2012,94.8
201,2,Stanford University,United States of America,94.8,57.2,98.9,99.8,63.8,93.9,"15,596",7.8,22.0,42 : 58,2012,93.9
202,2,Harvard University,United States of America,95.8,67.5,97.4,99.8,35.9,93.9,"20,152",8.9,25.0,NaN,2012,93.9
203,4,University of Oxford,United Kingdom,89.5,91.9,96.6,97.9,62.1,93.6,"19,919",11.6,34.0,46 : 54,2012,93.6
204,5,Princeton University,United States of America,91.5,49.6,99.1,100.0,81.0,92.9,"7,929",8.4,27.0,45 : 55,2012,92.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,350-400,Tokyo University of Agriculture and Technology,Japan,26.5,18.0,15.8,21.7,41.5,-,"5,865",14.4,5.0,27 : 73,2012,21.6
598,350-400,Hiroshima University,Japan,27.7,18.8,16.3,20.1,39.4,-,"14,686",8.2,5.0,36 : 64,2012,21.6
599,350-400,New Jersey Institute of Technology,United States of America,17.1,41.3,14.5,26.4,37.5,-,"10,646",26.2,17.0,25 : 75,2012,21.4
600,350-400,Michigan Technological University,United States of America,21.2,32.3,13.8,24.8,41.9,-,NaN,NaN,NaN,NaN,2012,21.4


In [59]:
times['group_index_times'] = times.groupby('year').cumcount() + 1

In [60]:
times['new_world_ranking_times'] = times['group_index_times'] 

In [61]:
for i in range(1, len(times)):
                
    if times.new_total_score.loc[i-1] != times.new_total_score.loc[i]:
        times['new_world_ranking_times'].loc[i] = times['group_index_times'].loc[i]
        
    else:
        times['new_world_ranking_times'].loc[i] = times.new_world_ranking_times.loc[i-1]
    i = i+1

C:\Users\valen\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
times

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25.0,NaN,2011,95.5,1,1
1,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33.0,37 : 63,2011,95.1,2,2
2,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27.0,33 : 67,2011,94.9,3,3
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22.0,42 : 58,2011,92.5,4,4
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,0.0,94.2,"7,929",8.4,27.0,45 : 55,2011,91.1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Southern Federal University,Russian Federation,21.9,17.3,8.2,9.2,39.5,-,"27,556",9.0,3.0,56 : 44,2016,14.1,796,792
2599,601-800,V.N. Karazin Kharkiv National University,Ukraine,21.7,48.4,8.9,1.7,28.8,-,"14,410",9.7,22.0,53 : 47,2016,14.0,797,797
2600,601-800,Federal University of Bahia,Brazil,18.2,19.7,8.1,14.3,0.0,-,"39,861",17.9,1.0,47 : 53,2016,13.7,798,798
2601,601-800,University of Brasília,Brazil,21.9,24.1,7.4,10.4,0.0,-,"45,880",14.5,2.0,52 : 48,2016,13.7,799,798


In [63]:
times.to_csv(r"C:\Users\valen\Desktop\times.csv")

In [64]:
times[times['world_rank'] == '1']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25.0,NaN,2011,95.5,1,1
200,1,California Institute of Technology,United States of America,95.7,56.0,98.2,99.9,97.0,94.8,"2,243",6.9,27.0,33 : 67,2012,94.8,1,1
602,1,California Institute of Technology,United States of America,96.3,59.8,99.4,99.7,95.6,95.5,"2,243",6.9,27.0,33 : 67,2013,95.5,1,1
1002,1,California Institute of Technology,United States of America,94.4,65.8,98.2,99.8,91.2,94.9,"2,243",6.9,27.0,33 : 67,2014,94.9,1,1
1402,1,California Institute of Technology,United States of America,92.2,67.0,98.1,99.7,89.1,94.3,"2,243",6.9,27.0,33 : 67,2015,94.3,1,1
1803,1,California Institute of Technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,"2,243",6.9,27.0,33 : 67,2016,95.1,1,1


In [65]:
#creazione del df con i dati più recenti per ciascuna università
times_most_recent = times.loc[times.groupby('university_name')['year'].idxmax()].reset_index()

In [66]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,2541,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,"35,569",17.0,1.0,-,2016,17.4,739,739
1,2051,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247
2,2059,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255
3,1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106
4,2143,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20.0,46 : 54,2016,69.0,54,54
814,2010,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14.0,49 : 51,2016,47.9,208,207
815,1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30.0,18 : 82,2016,57.9,102,101
816,1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54.0,27 : 73,2016,76.0,31,31


In [67]:
#ds cwur
#replacement dei caratteri speciali 
cwur['institution'] = cwur['institution'].str.replace('[^A-Za-z0-9]', ' ')

In [68]:
cwur

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Cear,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coru a,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [69]:
cwur.rename(columns={'institution':'university_name'}, inplace=True)

In [70]:
cwur.rename(columns={'world_rank':'world_rank_cwur'}, inplace=True)

In [71]:
cwur_most_recent = cwur.loc[cwur.groupby('university_name')['year'].idxmax()].reset_index()

In [72]:
merged_rankings = shanghai_most_recent.merge(cwur_most_recent,on=['university_name'], how = 'outer').merge(times_most_recent,on=['university_name'], how='outer')[['university_name','new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']]

In [73]:
merged_rankings

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
0,College of France,151.0,NaN,NaN
1,Northern Arizona University,457.0,NaN,766.0
2,Loyola University Chicago,475.0,NaN,437.0
3,Memorial University of Newfoundland,484.0,553.0,611.0
4,University of Wroclaw,484.0,NaN,NaN
...,...,...,...,...
1493,École Normale Supérieure,NaN,54.0,NaN
1494,École Normale Supérieure de Lyon,NaN,207.0,NaN
1495,École Polytechnique,NaN,101.0,NaN
1496,École Polytechnique Fédérale de Lausanne,NaN,31.0,NaN


In [74]:
merged_rankings = merged_rankings.fillna(0.00)

In [75]:
merged_rankings_match = merged_rankings[((merged_rankings['new_world_ranking_times']!= 0) & (merged_rankings['world_rank_cwur']!= 0))|((merged_rankings['world_rank_cwur'] != 0)&(merged_rankings['new_world_ranking_sha']!= 0))| ((merged_rankings['new_world_ranking_times'] != 0)&(merged_rankings['new_world_ranking_sha']!= 0))]

#merged_rankings_match = merged_rankings[(merged_rankings['new_world_ranking_sha']!= 0)&(merged_rankings['new_world_ranking_times']!= 0)&(merged_rankings['world_rank'] != 0)]
merged_rankings_match

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
1,Northern Arizona University,457.0,0.0,766.0
2,Loyola University Chicago,475.0,0.0,437.0
3,Memorial University of Newfoundland,484.0,553.0,611.0
5,Hacettepe University,490.0,729.0,634.0
7,University of Calcutta,498.0,661.0,891.0
...,...,...,...,...
1230,Xidian University,0.0,777.0,991.0
1234,Yeungnam University,0.0,701.0,770.0
1235,Yokohama City University,0.0,617.0,513.0
1236,Yokohama National University,0.0,737.0,518.0


In [76]:
merged_rankings_match['max_rank'] = merged_rankings_match[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']].max(axis=1)

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
merged_rankings_match['min_rank'] = merged_rankings_match[merged_rankings_match[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']] !=0].min(axis=1)

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
merged_rankings_match['diff_rank'] = merged_rankings_match['max_rank'] - merged_rankings_match['min_rank']

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
merged_rankings_match

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur,max_rank,min_rank,diff_rank
1,Northern Arizona University,457.0,0.0,766.0,766.0,457.0,309.0
2,Loyola University Chicago,475.0,0.0,437.0,475.0,437.0,38.0
3,Memorial University of Newfoundland,484.0,553.0,611.0,611.0,484.0,127.0
5,Hacettepe University,490.0,729.0,634.0,729.0,490.0,239.0
7,University of Calcutta,498.0,661.0,891.0,891.0,498.0,393.0
...,...,...,...,...,...,...,...
1230,Xidian University,0.0,777.0,991.0,991.0,777.0,214.0
1234,Yeungnam University,0.0,701.0,770.0,770.0,701.0,69.0
1235,Yokohama City University,0.0,617.0,513.0,617.0,513.0,104.0
1236,Yokohama National University,0.0,737.0,518.0,737.0,518.0,219.0


In [80]:
merged_rankings_match[merged_rankings_match['university_name']=='Aarhus University']

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur,max_rank,min_rank,diff_rank
228,Aarhus University,73.0,106.0,122.0,122.0,73.0,49.0


In [81]:
merged_rankings_match[merged_rankings_match['university_name']=='Aalborg University']

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur,max_rank,min_rank,diff_rank
547,Aalborg University,392.0,247.0,565.0,565.0,247.0,318.0


# **5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.**

In [82]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,2541,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,"35,569",17.0,1.0,-,2016,17.4,739,739
1,2051,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247
2,2059,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255
3,1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106
4,2143,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20.0,46 : 54,2016,69.0,54,54
814,2010,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14.0,49 : 51,2016,47.9,208,207
815,1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30.0,18 : 82,2016,57.9,102,101
816,1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54.0,27 : 73,2016,76.0,31,31


In [83]:
times_most_recent['fem_ratio'] = times_most_recent.female_male_ratio.str.extract(r'(.*):')
times_most_recent['male_ratio'] = times_most_recent.female_male_ratio.str.extract(r'.*:(.*)')

In [84]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio
0,2541,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,0.0,-,"35,569",17.0,1.0,-,2016,17.4,739,739,NaN,NaN
1,2051,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247,48,52
2,2059,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255,32,68
3,1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106,54,46
4,2143,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341,48,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20.0,46 : 54,2016,69.0,54,54,46,54
814,2010,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14.0,49 : 51,2016,47.9,208,207,49,51
815,1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30.0,18 : 82,2016,57.9,102,101,18,82
816,1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54.0,27 : 73,2016,76.0,31,31,27,73


In [85]:
times_most_recent = times_most_recent[times_most_recent['male_ratio'].notna()]

In [86]:
times_most_recent.dtypes

index                        int64
world_rank                  object
university_name             object
country                     object
teaching                   float64
international              float64
research                   float64
citations                  float64
income                     float64
total_score                 object
num_students                object
student_staff_ratio        float64
international_students     float64
female_male_ratio           object
year                         int64
new_total_score            float64
group_index_times            int64
new_world_ranking_times      int64
fem_ratio                   object
male_ratio                  object
dtype: object

In [87]:
times_most_recent['fem_ratio'] = times_most_recent['fem_ratio'].astype(int)
times_most_recent['male_ratio'] = times_most_recent['male_ratio'].astype(int)

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [88]:
times_most_recent['num_students'] = times_most_recent['num_students'].str.replace(',', '')
times_most_recent['num_students'] =times_most_recent['num_students'].astype(int)

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
times_most_recent['num_female'] = ( times_most_recent['fem_ratio']*times_most_recent['num_students']/100).round(0).astype(int)
times_most_recent['num_male'] = (times_most_recent['male_ratio']*times_most_recent['num_students']/100).round(0).astype(int)

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,...,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio,num_female,num_male
1,2051,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,...,15.0,48 : 52,2016,44.6,249,247,48,52,8363,9059
2,2059,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,...,17.0,32 : 68,2016,44.2,257,255,32,68,5152,10947
3,1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,...,14.0,54 : 46,2016,57.7,106,106,54,46,12903,10992
4,2143,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,18.0,48 : 52,2016,38.5,341,341,48,52,4441,4811
5,2555,601-800,Adam Mickiewicz University,Poland,20.0,25.7,11.0,15.3,28.7,-,...,1.0,71 : 29,2016,16.5,753,753,71,29,28849,11784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,...,20.0,46 : 54,2016,69.0,54,54,46,54,1104,1296
814,2010,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,...,14.0,49 : 51,2016,47.9,208,207,49,51,1087,1131
815,1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,...,30.0,18 : 82,2016,57.9,102,101,18,82,437,1992
816,1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,...,54.0,27 : 73,2016,76.0,31,31,27,73,2610,7056


In [91]:
female_country = times_most_recent.groupby('country')['num_female'].sum()
male_country = times_most_recent.groupby('country')['num_male'].sum()
FM_Country = pd.merge(female_country, male_country, on=['country'])

In [92]:
FM_Country

,num_female,num_male
country,,
Argentina,67191,41182
Australia,391736,321640
Austria,68364,66113
Bangladesh,21323,41393
Belarus,20219,9084
...,...,...
Uganda,18670,18670
Ukraine,17846,19250
United Arab Emirates,9516,4931


# **6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)**

In [93]:
times_most_recent['ratio_']= times_most_recent['fem_ratio']/times_most_recent['male_ratio']

C:\Users\valen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
times_most_recent = times_most_recent[times_most_recent['male_ratio']!=0]

In [95]:
media = times_most_recent['ratio_'].mean()

In [96]:
ratio_inf=times_most_recent[times_most_recent['ratio_']<times_most_recent['ratio_'].mean()]

In [97]:
media

1.0811020920897247

In [98]:
len(ratio_inf)

346

In [99]:
ratio_inf

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,...,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio,num_female,num_male,ratio_
1,2051,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,...,48 : 52,2016,44.6,249,247,48,52,8363,9059,0.923077
2,2059,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,...,32 : 68,2016,44.2,257,255,32,68,5152,10947,0.470588
4,2143,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,48 : 52,2016,38.5,341,341,48,52,4441,4811,0.923077
7,2510,601-800,Ajou University,South Korea,19.5,20.0,11.9,23.9,45.7,-,...,33 : 67,2016,19.2,708,706,33,67,4193,8513,0.492537
8,2561,601-800,Alexandria University,Egypt,20.3,33.7,8.2,14.1,29.7,-,...,46 : 54,2016,16.0,759,758,46,54,58618,68813,0.851852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,2086,251-300,Zhejiang University,China,44.3,19.6,46.0,36.1,96.2,-,...,41 : 59,2016,41.8,284,284,41,59,19478,28030,0.694915
813,1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,...,46 : 54,2016,69.0,54,54,46,54,1104,1296,0.851852
814,2010,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,...,49 : 51,2016,47.9,208,207,49,51,1087,1131,0.960784
815,1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,...,18 : 82,2016,57.9,102,101,18,82,437,1992,0.219512


# **7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country)**

In [100]:
ratio_inf_country = ratio_inf.groupby('country')['num_students'].sum().reset_index(name ='Sum_inf_avg_ratio')
ratio_inf_country 

,country,Sum_inf_avg_ratio
0,Australia,146235
1,Austria,61033
2,Bangladesh,62716
3,Brazil,278774
4,Canada,86779
5,Chile,90951
6,China,885293
7,Colombia,54645
8,Czech Republic,38685
9,Denmark,45705


In [101]:
times_most_recent_grouped = times_most_recent.groupby('country')['num_students'].sum().reset_index(name ='Sum_total_country')
times_most_recent_grouped

,country,Sum_total_country
0,Argentina,108373
1,Australia,713376
2,Austria,134477
3,Bangladesh,62716
4,Belarus,29303
...,...,...
65,Uganda,37340
66,Ukraine,37096
67,United Arab Emirates,14447
68,United Kingdom,1324842


In [102]:
merged = pd.merge(times_most_recent_grouped, ratio_inf_country, on= 'country')

In [103]:
merged['prop_inf_avg'] = merged['Sum_inf_avg_ratio']/merged['Sum_total_country']
merged

,country,Sum_total_country,Sum_inf_avg_ratio,prop_inf_avg
0,Australia,713376,146235,0.204990
1,Austria,134477,61033,0.453855
2,Bangladesh,62716,62716,1.000000
3,Brazil,494251,278774,0.564033
4,Canada,623629,86779,0.139152
5,Chile,90951,90951,1.000000
6,China,1015150,885293,0.872081
7,Colombia,54645,54645,1.000000
8,Czech Republic,175871,38685,0.219962
9,Denmark,119355,45705,0.382933


# **8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name**

In [104]:
attainment= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\educational_attainment_supplementary_data.csv')

In [105]:
attainment.isnull()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
1,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
2,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
3,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
4,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79051,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79052,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79053,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [106]:
attainment = attainment[attainment['country_name'].notna()]

In [107]:
attainment = attainment[attainment['series_name'].notna()]

In [108]:
attainment[attainment['country_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


In [109]:
attainment[attainment['series_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


# **9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value**

In [110]:
attainment.head(5)

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [111]:
attainment_mod = attainment.melt(id_vars=["country_name", "series_name"], var_name="Year", value_name="Value")

In [112]:
attainment_mod

,country_name,series_name,Year,Value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


# **10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).**

In [113]:
shanghai_new = shanghai['university_name'].unique()

In [114]:
times_new = times['university_name'].unique()

In [115]:
cwur_new = cwur['university_name'].unique()

In [116]:
rank_list = [shanghai_new, cwur_new, times_new]  
concat_rankings = np.concatenate(rank_list)


In [117]:
occorrenze = collections.Counter(concat_rankings)
occorrenze

Counter({'Harvard University': 3,
         'University of Cambridge': 3,
         'Stanford University': 3,
         'University of California  Berkeley': 2,
         'Massachusetts Institute of Technology  MIT ': 1,
         'California Institute of Technology': 3,
         'Columbia University': 3,
         'Princeton University': 3,
         'University of Chicago': 3,
         'University of Oxford': 3,
         'Yale University': 3,
         'Cornell University': 3,
         'University of California  San Diego': 2,
         'University of California  Los Angeles': 2,
         'University of Pennsylvania': 3,
         'University of Wisconsin   Madison': 1,
         'University of Washington': 2,
         'University of California  San Francisco': 2,
         'The Johns Hopkins University': 1,
         'The University of Tokyo': 1,
         'University of Michigan   Ann Arbor': 1,
         'Kyoto University': 3,
         'The Imperial College of Science  Technology and Medicine': 

# **11. In the times ranking, compute the number of times each university appears**

In [120]:
rank_univ_times = times.groupby('university_name')['year'].count().reset_index(name = 'count')
rank_univ_times

,university_name,count
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


# **12. Find the universities that appear at most twice in the times ranking.**

In [121]:
rank_univ_times[rank_univ_times['count']<=2]

,university_name,count
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


# **13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).**

In [122]:
shanghai_n = shanghai[['university_name', 'year', 'new_world_ranking_sha']]

In [123]:
times_n = times[['university_name', 'year', 'new_world_ranking_times']]

In [124]:
cwur_n = cwur[['university_name', 'year', 'world_rank_cwur']]

In [125]:
merged_rankings_n = shanghai_n.merge(cwur_n,on=['university_name', 'year'], how = 'outer').merge(times_n,on=['university_name', 'year'], how='outer')[['university_name','year','new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']]


In [126]:
merged_rankings_n = merged_rankings_n.fillna(0)

In [127]:
merged_rankings_match_n = merged_rankings_n[(merged_rankings_n['new_world_ranking_sha']!= 0)&(merged_rankings_n['new_world_ranking_times']!= 0)&(merged_rankings_n['world_rank_cwur'] != 0)]

In [128]:
merged_rankings_match_n

,university_name,year,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
3514,Harvard University,2012,1.0,2.0,1.0
3515,Stanford University,2012,2.0,2.0,3.0
3518,University of Cambridge,2012,5.0,6.0,4.0
3519,California Institute of Technology,2012,6.0,1.0,5.0
3520,Princeton University,2012,7.0,5.0,6.0
...,...,...,...,...,...
4861,Ewha Womans University,2015,465.0,363.0,390.0
4874,University of Graz,2015,479.0,354.0,584.0
4876,Sharif University of Technology,2015,479.0,305.0,697.0
4877,Syracuse University,2015,479.0,176.0,463.0


In [129]:
same_rank = merged_rankings_n[merged_rankings_n[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']].nunique(axis=1) == 1]
same_rank

,university_name,year,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
3799,Stanford University,2013,2.0,2.0,2.0
